# Mosquitoes, beer, before, after

This is an example of merging in the familiar [mosquito
beer](https://github.com/matthew-brett/datasets/tree/master/mosquito_beer)
dataset.  Please also look at the [the data license page](data/LICENSE).

If you want to run this notebook on your own computer, Download the data from
{download}`mosquito_beer.csv <data/mosquito_beer.csv>`.

In [ ]:
# Import Numpy library, rename as "np"
import numpy as np
# Make random number generator.
rng = np.random.default_rng()
# Import Pandas library, rename as "pd"
import pandas as pd
# Safe setting for Pandas.
pd.set_option('mode.copy_on_write', True)

# Set up plotting
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

As you may remember, the setup for the experiment was the following:

* There were 43 volunteers who took part in the experiment.
* The authors allocated 25 volunteers to the "beer" group, and 18
  to the "water" group.
* Each volunteer had two trails, a "before" trial, and an "after"
  trial.
* The "before" trial was a trial before they drank their
  allocated drink of beer or water.
* The "after" trial was after they drank their allocated drink of
  beer or water.
* In a trial the volunteers sat in tent.  There was a plastic
  tube connecting the tent to a closed box containing 50
  mosquitoes.  The trial started when the experimenters opened
  the box of mosquitoes, allowing them to fly down the tube
  towards the smell of person in the tent.
* The "activated" number is the number, out of 50, of mosquitoes
  that flew out of the box into the plastic tube.

In [ ]:
mosquitoes = pd.read_csv('data/mosquito_beer.csv')
mosquitoes.head()

We have two "activated" numbers per volunteer, one "before" and
one "after".   If we subtract "before" from "after" we get a
measure of the *difference* caused by the drink.

We might want to do this because we speculate that different
people may differ in how attractive they are mosquitoes.  If we
just look, for example, at the "after" numbers, there will be
some number of mosquitoes that can be explained by the person's
own odor, regardless of drink, and some number of mosquitoes explained by the change in odor caused by the drink.

By subtracting "before" from "after" we have a measure of the difference due to
the drink, and we've subtracted away any difference due to the person.

In order to do this subtraction, we have to define the values to subtract.

We get the rows corresponding to the "before" and "after" phases of the
experiment.  There is one "before" row per subject (`volunteer`) and one
"after" row.  We need the `activated` numbers from those rows.

In [ ]:
before_activated = mosquitoes.loc[mosquitoes['test'] == 'before',
                                 ['volunteer', 'group', 'activated']]
before_activated.head()

In [ ]:
after_activated = mosquitoes.loc[mosquitoes['test'] == 'after',
                                 ['volunteer', 'group', 'activated']]
after_activated.head()

We would like to merge the before and after DataFrames, so we can match up the before and after `activated` values, before subtracting.

We could `merge` on `volunteer`, to do this:

In [ ]:
before_activated.merge(after_activated, on='volunteer')

Notice the rather ugly column labels Pandas generates because the two
DataFrames have the same column names, and Pandas needs to distinguish between
them.

We can make better suffixes for the column names, to give the merged columns
nicer names:

In [ ]:
both = before_activated.merge(after_activated, on='volunteer',
                              suffixes=['_before', '_after'])
both.head()

Now we have matched up the "before" and "after" rows, we can do the
subtraction:

In [ ]:
both['active_diff'] = both['activated_after'] - both['activated_before']
both.head()

We are expecting the `group` to be the same for the "before" and "after" rows, because each `volunteer` only belonged to one group:

In [ ]:
# Raise an error if the group_before and group_after values differ in any row.
# We expect no error.
assert np.all(both['group_before'] == both['group_after'])

To make the DataFrame easier to read, we rename the `group_before` to just
`group` and drop the `group_before`, `group_after` columns.

In [ ]:
both['group'] = both['group_before']
both = both.drop(columns=['group_before', 'group_after'])
both.head()

We can then get the means of the `active_diff` subtracted values per group, and calculated the difference in means.  This is our *observed statistic*.

In [ ]:
means = both.groupby('group')['active_diff'].mean()
observed_stat = means.loc['beer'] - means.loc['water']
observed_stat

Now we get ready to simulate in the null-world, using the same method as we did
in [mosquitoes and labels](mosquito_labels.Rmd):


In [ ]:
# The labels column we will permute.
observed_groups = both['group']
# The null-world DataFrame.
fake_df = both.copy()

The procedure for one trial (simulation):

In [ ]:
fake_df['group'] = rng.permutation(observed_groups)
fake_means = fake_df.groupby('group')['active_diff'].mean()
fake_stat = fake_means['beer'] - fake_means['water']
fake_stat

Building up the sampling distribution with many simulations:

In [ ]:
n_iters = 10000
fake_stats = np.zeros(n_iters)
for i in np.arange(n_iters):
    fake_df['group'] = rng.permutation(observed_groups)
    fake_means = fake_df.groupby('group')['active_diff'].mean()
    fake_stats[i] = fake_means['beer'] - fake_means['water']
fake_stats[:10]

In [ ]:
plt.hist(fake_stats, bins=50);
plt.plot(observed_stat, 10, 'ro')
plt.title("Sampling distribution of mean difference");

The chance of seeing the observed statistic, or larger, in the sampling
distribution, is small — we do have reason to doubt the null-model.

In [ ]:
p = np.count_nonzero(fake_stats >= observed_stat) / n_iters
p